In [ ]:
pip install pandas numpy scikit-learn matplotlib seaborn

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit

In [4]:
# Load data
df = pd.read_csv('daily_aggregated.csv')

# Define features
EXCLUDE_COLS = ['date', 'daily_ridership', 'state', 'mode_train', 'mode_subway']
numeric_cols = df.select_dtypes(include='number').columns
ALL_FEATURES = [col for col in numeric_cols if col not in EXCLUDE_COLS]
LOG_FEATURES = ['rain_sum', 'rain_max', 'snowfall_sum', 'snowfall_max',
                'us_aqi_pm2_5_mean', 'us_aqi_pm10_mean', 'us_aqi_sulphur_dioxide_mean']

# Get all city/mode combinations
subsets = df[['state', 'mode']].drop_duplicates()

In [18]:
# Loop through each subset

results = []

# Loop through subsets
for _, row in subsets.iterrows():
    state, mode = row['state'], row['mode']
    print(f"Training Lasso for {state} - {mode}...")

    # Filter and sort by date
    df_sub = df[(df['state'] == state) & (df['mode'] == mode)].copy()
    df_sub = df_sub.dropna(subset=ALL_FEATURES + ['daily_ridership'])
    df_sub['date'] = pd.to_datetime(df_sub['date'])
    df_sub = df_sub.sort_values('date')

    # Log transform target
    df_sub['daily_ridership'] = np.log1p(df_sub['daily_ridership'])

    # Log transform skewed inputs
    for col in LOG_FEATURES:
        if col in df_sub.columns:
            df_sub[col] = np.log1p(df_sub[col])

    X = df_sub[ALL_FEATURES]
    y = df_sub['daily_ridership']

    # Time based split
    # https://stackoverflow.com/questions/52138064/how-to-split-into-train-test-and-cv-in-an-non-reshuffle-order
    # https://apxml.com/courses/time-series-analysis-forecasting/chapter-6-model-evaluation-selection/train-test-split-time-series
    split_idx = int(len(df_sub) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]

    # Grid search with time based CV
    pipeline = make_pipeline(StandardScaler(), Lasso(max_iter=10000))
    param_grid = {'lasso__alpha': np.logspace(-4, 1, 50)}
    tscv = TimeSeriesSplit(n_splits=5, test_size=int(len(X_train) * 0.1))
    grid = GridSearchCV(pipeline, param_grid, scoring='r2', cv=tscv)
    grid.fit(X_train, y_train)
    best_model = grid.best_estimator_

    # Cross validation metrics (training set)
    r2_cv = cross_val_score(best_model, X_train, y_train, cv=tscv, scoring='r2')

    # Final evaluation (test set)
    y_pred = best_model.predict(X_test)
    y_test_actual = np.expm1(y_test)
    y_pred_actual = np.expm1(y_pred)

    r2_test = r2_score(y_test_actual, y_pred_actual)
    rmse_test = np.sqrt(mean_squared_error(y_test_actual, y_pred_actual))
    mae_test = mean_absolute_error(y_test_actual, y_pred_actual)

    # Results
    results.append({
        "state": state,
        "Mode": mode,
        "CV R² (mean)": round(r2_cv.mean(), 4),
        "CV R² (std)": round(r2_cv.std(), 4),
        "Test R²": round(r2_test, 4),
        "Test RMSE": round(rmse_test, 2),
        "Test MAE": round(mae_test, 2)
    })

# Display summary table
results_df = pd.DataFrame(results)
print("Summary of Lasso Results by Subset:")
display(results_df)


Training Lasso for NYC - bus...
Training Lasso for NYC - subway...
Training Lasso for CHI - bus...
Training Lasso for CHI - train...
Summary of Lasso Results by Subset:


,state,Mode,CV R² (mean),CV R² (std),Test R²,Test RMSE,Test MAE
0,NYC,bus,0.7729,0.0958,0.7521,151071.30,130709.11
1,NYC,subway,0.7846,0.0916,0.6466,503014.05,429736.27
2,CHI,bus,0.6730,0.0257,0.6690,76121.75,65179.51
3,CHI,train,0.6147,0.0824,0.5183,52244.50,41803.62


In [20]:
# Define air quality features
aq_features = [col for col in ALL_FEATURES if 'aqi' in col]

# Collect readable interpretation statements
interpretation_rows = []

# Re run loop for each subset
for _, row in subsets.iterrows():
    state, mode = row['state'], row['mode']
    print(f"Working on feature impact for {state} - {mode}")

    # Filter data
    df_sub = df[(df['state'] == state) & (df['mode'] == mode)].copy()
    df_sub = df_sub.dropna(subset=ALL_FEATURES + ['daily_ridership'])

    df_sub['daily_ridership'] = np.log1p(df_sub['daily_ridership'])

    for col in LOG_FEATURES:
        if col in df_sub.columns:
            df_sub[col] = np.log1p(df_sub[col])

    X = df_sub[ALL_FEATURES]
    y = df_sub['daily_ridership']

    # Time based split
    split_idx = int(len(df_sub) * 0.8)
    
    X_train = X.iloc[:split_idx]
    X_test = X.iloc[split_idx:]
    y_train = y.iloc[:split_idx]
    y_test = y.iloc[split_idx:]

    # Fit model again to extract components
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    lasso = Lasso(alpha=grid.best_params_['lasso__alpha'], max_iter=10000)
    lasso.fit(X_train_scaled, y_train)

    coefs = pd.Series(lasso.coef_, index=ALL_FEATURES)
    aq_coefs = coefs[coefs.index.isin(aq_features)]

    top_feature = aq_coefs.abs().idxmax()
    top_coef = aq_coefs[top_feature]
    feature_std = scaler.scale_[ALL_FEATURES.index(top_feature)]

    median_log_ridership = np.log1p(df_sub['daily_ridership']).median()
    y_pred_baseline = np.expm1(median_log_ridership)
    y_pred_shifted = np.expm1(median_log_ridership + top_coef * feature_std)
    delta_riders = y_pred_shifted - y_pred_baseline

    interpretation_rows.append({
        'state': state,
        'mode': mode,
        'top_aq_feature': top_feature,
        'impact': f"{'+' if delta_riders > 0 else '-'} ~{abs(int(delta_riders)):,} riders"
    })

# Display summary table
impact_df = pd.DataFrame(interpretation_rows)
display(impact_df)

Working on feature impact for NYC - bus
Working on feature impact for NYC - subway
Working on feature impact for CHI - bus
Working on feature impact for CHI - train


,state,mode,top_aq_feature,impact
0,NYC,bus,us_aqi_nitrogen_dioxide_max,+ ~4 riders
1,NYC,subway,us_aqi_max,- ~1 riders
2,CHI,bus,us_aqi_sulphur_dioxide_mean,- ~0 riders
3,CHI,train,us_aqi_max_bin,- ~0 riders
